In [33]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
# 001
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K

# 002
# from keras._tf_keras.keras.preprocessing.image import  ImageDataGenerator
# from keras._tf_keras.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
# from keras._tf_keras.keras import models, layers, optimizers, callbacks
# from keras._tf_keras.keras import backend as K


print("Todas as bibliotecas foram importadas com sucesso!")

Todas as bibliotecas foram importadas com sucesso!


In [34]:
# Criando função para calcular as métricas

def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)  
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }


In [35]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    else:
        raise ValueError("Escala de cor não suportada.")

In [36]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor

def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário
                img = convert_color_scale(img, color_scale)  # Converta para a escala de cor desejada
                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal
                images.append(img)

    return np.array(images)


In [37]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"

normal_dir = os.path.join(data_dir, 'Non_Cancerous')
endometri = os.path.join(data_dir, 'Endometri')
Clear_Cell = os.path.join(data_dir, 'Clear_Cell')
Mucinous = os.path.join(data_dir, 'Mucinous')
Serous = os.path.join(data_dir, 'Serous')

img_extensions = ['.jpg', '.jpeg', '.png']

if os.listdir(normal_dir):print("pasta acesada, na pasta normal ha itens")
if os.listdir(endometri):print("pasta acessada, na pasta endometri ha itens")
if os.listdir(Clear_Cell):print("pasta acessada, na pasta Clear_Cell ha itens")
if os.listdir(Mucinous):print("pasta acessada, na pasta Mucinous ha itens")
if os.listdir(Serous):print("pasta acessada, na pasta Serous ha itens")



pasta acesada, na pasta normal ha itens
pasta acessada, na pasta endometri ha itens
pasta acessada, na pasta Clear_Cell ha itens
pasta acessada, na pasta Mucinous ha itens
pasta acessada, na pasta Serous ha itens


In [38]:
# Criar um DataFrame para armazenar os resultados
columns = ['Modelo', 'Acuracia', 'Sensibilidade', 'Especificidade', 'F-Score', 'AUC-ROC']
df_metrics = pd.DataFrame(columns=columns)

# Definir o caminho para salvar o melhor modelo
model_checkpoint_path = r"C:\Users\andre\Documents\atividades\estudos_opencv\introd_processamento de imagens\Aplicando_CNN_selection\\CNN-best_model.h5"

In [39]:
# Iterar sobre os modelos
def model_execution(model_name,pasta_com_nonCancer):

    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(normal_dir, 'xyz',img_extensions)
    cancer_images = load_images(pasta_com_nonCancer, 'xyz',img_extensions)

    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    # Escolha entre 5 tipos de conversão de escala de cor aqui (por exemplo, normalização)
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Escolher o modelo de CNN
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VisionTransformer':
        base_model = VitB16(image_size=(224, 224), weights='imagenet', include_top=False)
    else:
        raise ValueError("Modelo de CNN não suportado.")

    # Construir o modelo
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compilar o modelo
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Configurar o retorno de chamada ModelCheckpoint
    checkpoint = callbacks.ModelCheckpoint(model_checkpoint_path,
                                           monitor='val_accuracy',  # Métrica a ser monitorad
                                           mode='max',               # Salvar o modelo com a maior precisão
                                           verbose=1)

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)  #  16 <<====

    # Avaliar o modelo
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas
    metrics = get_metrics(y_test, y_pred_binary)

    # Adicionar as métricas ao DataFrame
    metrics['Modelo'] = model_name
    
    # Excluir o modelo atual para liberar memória da GPU   
    del all_images, normal_images, cancer_images, X_train, X_test, y_train, y_test, y_pred, y_pred_binary
    del all_labels, normal_labels, cancer_labels
    tf.keras.backend.clear_session()
    del model, base_model
    gc.collect()
    
    return metrics


In [40]:
chosen_model = 'VGG16'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics, temp_df], ignore_index=True) # <==



chosen_model = 'ResNet50'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'InceptionV3'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'MobileNetV2'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



chosen_model = 'EfficientNetB0'
#       ENDOMETRI
metrics = model_execution(chosen_model,endometri)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics1 = pd.concat([df_metrics1, temp_df], ignore_index=True)
#       Clear_Cell  
metrics = model_execution(chosen_model,Clear_Cell) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics2 = pd.concat([df_metrics2, temp_df], ignore_index=True) # <==
#       Mucinous
metrics = model_execution(chosen_model,Mucinous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics3 = pd.concat([df_metrics3, temp_df], ignore_index=True) # <==
#       Serous
metrics = model_execution(chosen_model,Serous) # <==
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics4 = pd.concat([df_metrics4, temp_df], ignore_index=True) # <==



print("\033[32m-------    ENDOMETRI e NON_CONCEROUS   ------\033[m ")
display(df_metrics1)

print("\033[32m-------    Clear_Cell e NON_CONCEROUS   ------\033[m ")
display(df_metrics2)

print("\033[32m-------    Mucinous e NON_CONCEROUS   ------\033[m ")
display(df_metrics3)

print("\033[32m-------    Serous e NON_CONCEROUS   ------\033[m ")
display(df_metrics4)


# # Adicionar identificadores
# df_metrics1['Identificador'] = 'ENDOMETRI e NON_CONCEROUS'
# df_metrics2['Identificador'] = 'Clear_Cell e NON_CONCEROUS'
# df_metrics3['Identificador'] = 'Mucinous e NON_CONCEROUS'
# df_metrics4['Identificador'] = 'Serous e NON_CONCEROUS'

# # Adicionar linhas vazias entre os DataFrames
# empty_row = pd.DataFrame([[''] * len(df_metrics1.columns)], columns=df_metrics1.columns)

# # Concatenar DataFrames com linhas vazias entre eles
# combined_df = pd.concat([
#     df_metrics1, empty_row, 
#     df_metrics2, empty_row, 
#     df_metrics3, empty_row, 
#     df_metrics4
# ], ignore_index=True)

# # Salvar no Excel
# combined_df.to_excel('CNN-combined_metrics_HSV.xlsx', index=False, sheet_name='Metrics')


Epoch 1/50
8/8 [==============================] - 2s 166ms/step - loss: 0.7689 - accuracy: 0.4640 - val_loss: 0.6632 - val_accuracy: 0.5938
Epoch 2/50
8/8 [==============================] - 1s 146ms/step - loss: 0.6809 - accuracy: 0.5920 - val_loss: 0.6661 - val_accuracy: 0.4688
Epoch 3/50
8/8 [==============================] - 1s 145ms/step - loss: 0.6899 - accuracy: 0.5840 - val_loss: 0.6346 - val_accuracy: 0.5938
Epoch 4/50
8/8 [==============================] - 1s 144ms/step - loss: 0.6798 - accuracy: 0.5920 - val_loss: 0.5947 - val_accuracy: 0.8750
Epoch 5/50
8/8 [==============================] - 1s 143ms/step - loss: 0.5217 - accuracy: 0.7840 - val_loss: 0.3341 - val_accuracy: 0.9062
Epoch 6/50
8/8 [==============================] - 1s 145ms/step - loss: 0.2541 - accuracy: 0.9200 - val_loss: 0.2720 - val_accuracy: 0.9062
Epoch 7/50
8/8 [==============================] - 1s 144ms/step - loss: 0.2479 - accuracy: 0.9120 - val_loss: 0.1520 - val_accuracy: 0.9375
Epoch 8/50
8/8 [====

c:\Users\andre\anaconda3\envs\TFenv\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


Epoch 1/50
8/8 [==============================] - 5s 207ms/step - loss: 0.8779 - accuracy: 0.7165 - val_loss: 1.4170 - val_accuracy: 0.4375
Epoch 2/50
8/8 [==============================] - 1s 125ms/step - loss: 0.4291 - accuracy: 0.9449 - val_loss: 2.3536 - val_accuracy: 0.4375
Epoch 3/50
8/8 [==============================] - 1s 126ms/step - loss: 0.0259 - accuracy: 0.9921 - val_loss: 1.9073 - val_accuracy: 0.4375
Epoch 4/50
8/8 [==============================] - 1s 126ms/step - loss: 0.0510 - accuracy: 0.9685 - val_loss: 3.0946 - val_accuracy: 0.4375
Epoch 5/50
8/8 [==============================] - 1s 126ms/step - loss: 0.0516 - accuracy: 0.9843 - val_loss: 2.4958 - val_accuracy: 0.4375
Epoch 6/50
8/8 [==============================] - 1s 127ms/step - loss: 0.0328 - accuracy: 0.9843 - val_loss: 4.1369 - val_accuracy: 0.4375
Epoch 7/50
8/8 [==============================] - 1s 128ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 3.8995 - val_accuracy: 0.4375
Epoch 8/50
8/8 [====

c:\Users\andre\anaconda3\envs\TFenv\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


Epoch 1/50
8/8 [==============================] - 5s 205ms/step - loss: 1.6161 - accuracy: 0.5984 - val_loss: 1.5352 - val_accuracy: 0.4375
Epoch 2/50
8/8 [==============================] - 1s 125ms/step - loss: 0.1543 - accuracy: 0.9370 - val_loss: 1.1313 - val_accuracy: 0.4375
Epoch 3/50
8/8 [==============================] - 1s 125ms/step - loss: 0.1796 - accuracy: 0.9843 - val_loss: 0.8647 - val_accuracy: 0.4375
Epoch 4/50
8/8 [==============================] - 1s 126ms/step - loss: 0.0156 - accuracy: 0.9921 - val_loss: 1.0131 - val_accuracy: 0.4375
Epoch 5/50
8/8 [==============================] - 1s 127ms/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 1.1910 - val_accuracy: 0.4375
Epoch 6/50
8/8 [==============================] - 1s 126ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 1.2271 - val_accuracy: 0.4375
Epoch 7/50
8/8 [==============================] - 1s 127ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 1.1338 - val_accuracy: 0.4375
Epoch 8/50
8/8 [====

...

,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.825,0.789474,0.857143,0.810811,0.823308
1,ResNet50,0.525,0.000000,1.000000,0.000000,0.500000
2,InceptionV3,0.775,0.736842,0.809524,0.756757,0.773183
3,MobileNetV2,0.675,0.631579,0.714286,0.648649,0.672932
4,EfficientNetB0,0.500,1.000000,0.047619,0.655172,0.523810


-------    Mucinous e NON_CONCEROUS   ------ 


,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.700,0.736842,0.666667,0.700000,0.701754
1,ResNet50,0.475,1.000000,0.000000,0.644068,0.500000
2,InceptionV3,0.650,0.315789,0.952381,0.461538,0.634085
3,MobileNetV2,0.650,0.263158,1.000000,0.416667,0.631579
4,EfficientNetB0,0.525,0.105263,0.904762,0.173913,0.505013


-------    Serous e NON_CONCEROUS   ------ 


,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.850,0.842105,0.857143,0.842105,0.849624
1,ResNet50,0.475,1.000000,0.000000,0.644068,0.500000
2,InceptionV3,0.925,0.842105,1.000000,0.914286,0.921053
3,MobileNetV2,0.925,0.842105,1.000000,0.914286,0.921053
4,EfficientNetB0,0.450,0.210526,0.666667,0.266667,0.438596
